# Ontologie finale

In [1]:
# Ontologie Finale 

from owlready2 import get_ontology, Thing, DataProperty, ObjectProperty, FunctionalProperty
from unidecode import unidecode
import json

# Charger les données JSON
with open('./data/pokemons.json', 'r', encoding='utf-8') as file:
    pokemons_data = json.load(file)

# Créer une nouvelle ontologie
onto = get_ontology("http://example.org/onto/pokemon")

with onto:
    class Pokemon(Thing):
        pass

    class Type(Thing):
        pass
    
    class Stat(Thing):
        pass
    
    class Sensi(Thing):
        pass

    class Attack(Thing):
        pass

    class hasType(ObjectProperty):
        domain = [Pokemon]
        range = [Type]
    
    class hasStat(ObjectProperty):
        domain = [Pokemon]
        range = [Stat]
        
    class hasSensi(ObjectProperty):
        domain = [Pokemon]
        range = [Sensi]

    class hasAttack(ObjectProperty):
        domain = [Pokemon]
        range = [Attack]
        
    class nom(DataProperty, FunctionalProperty):
         domain = [Pokemon, Type, Stat, Sensi, Attack]
         range = [str]

    class valeur(DataProperty, FunctionalProperty): 
        domain = [Stat, Sensi]
        range = [float]
    
    class hasSensiType(ObjectProperty, FunctionalProperty):
        domain = [Sensi]
        range = [Type]
        
    class hasAttackType(ObjectProperty):
        domain = [Attack]
        range = [Type]
    
    class categorie(DataProperty, FunctionalProperty):
        domain = [Attack]
        range = [str]

    class puissance(DataProperty, FunctionalProperty):
        domain = [Attack]
        range = [int]

    class precision(DataProperty, FunctionalProperty):
        domain = [Attack]
        range = [int]

    class pp(DataProperty, FunctionalProperty):
        domain = [Attack]
        range = [int]

    # Pré-remplir les types pour réutilisation
    # Pré-remplir les types pour réutilisation, y compris un type "Inconnu"
    types = {name: Type(nom=name) for name in ["Normal", "Plante", "Feu", "Eau", "Electrik", "Glace", "Combat", "Poison", "Sol", "Vol", "Psy", "Insecte", "Roche", "Spectre", "Dragon", "Tenebres", "Acier", "Fee"]}
    types["Inconnu"] = Type(nom="Inconnu")  # Ajout du type "Inconnu" pour gérer les cas où le type est absent

    stats_cache = {}
    sensi_cache = {}
    attacks = {}
    attack_names = {}

    # Gérer la création des Pokémons et leurs caractéristiques
    for pokemon_data in pokemons_data:
        pokemon_nom = unidecode(pokemon_data['nom']) 
        pokemon_instance = Pokemon(nom=pokemon_nom)

        # Attribuer les types aux Pokémon
        for type_name in pokemon_data['types']:
            pokemon_instance.hasType.append(types[unidecode(type_name)])

        # Gérer les statistiques
        for stat_name, stat_value in pokemon_data['stats'].items():
            key = (unidecode(stat_name), float(stat_value))
            if key not in stats_cache:
                stats_cache[key] = Stat(nom=key[0], valeur=key[1])
            pokemon_instance.hasStat.append(stats_cache[key])
            
        # Gérer les sensibilités
        for sensi_name, sensi_value in pokemon_data['sensibilities'].items():
            # Clé pour le cache basée uniquement sur le type (sans nom)
            type_ref = types[unidecode(sensi_name)]
            key = (type_ref, float(sensi_value))  
            if key not in sensi_cache:
                sensi_instance = Sensi(valeur=key[1])
                sensi_instance.hasSensiType = type_ref  
                sensi_cache[key] = sensi_instance
            pokemon_instance.hasSensi.append(sensi_cache[key])

        # Attaques du Pokémon
        for attack_data in pokemon_data['attaques']:
            # Définir les valeurs par défaut pour les données manquantes
            attack_categorie = unidecode(attack_data['Catégorie']) if attack_data['Catégorie'] else "Inconnu"
            attack_puissance = attack_data['Puissance'] if attack_data['Puissance'] is not None else 0
            attack_precision = attack_data['Précision'] if attack_data['Précision'] is not None else 0
            attack_pp = attack_data['PP'] if attack_data['PP'] is not None else 0

            # Clé du cache basée sur les attributs fonctionnels essentiels, sans le type
            attack_key = (attack_categorie, attack_puissance, attack_precision, attack_pp)

            if attack_key not in attacks:
                attack_instance = Attack(
                    nom=unidecode(attack_data['Nom']),
                    categorie=attack_categorie,
                    puissance=attack_puissance,
                    precision=attack_precision,
                    pp=attack_pp
                )
                attack_type_name = unidecode(attack_data['Type']) if attack_data['Type'] else "Inconnu"
                attack_instance.hasAttackType.append(types[attack_type_name])
                attacks[attack_key] = attack_instance
                # Ajouter le nom de l'attaque au dictionnaire des noms
                attack_names[unidecode(attack_data['Nom'])] = attack_instance
            else:
                attack_instance = attacks[attack_key]
                # Enregistrer les noms alternatifs pour la même attaque
                if unidecode(attack_data['Nom']) not in attack_names:
                    attack_names[unidecode(attack_data['Nom'])] = attack_instance

            pokemon_instance.hasAttack.append(attack_instance)



            
    onto.save(file="pokemon_ontology.owl", format="rdfxml")
    print("Ontology created and saved.")


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Ontology created and saved.


In [1]:
# from owlready2 import get_ontology, Thing, DataProperty, ObjectProperty, FunctionalProperty
# from unidecode import unidecode
# import json

# # Charger les données JSON
# with open('./data/pokemons.json', 'r', encoding='utf-8') as file:
#     pokemons_data = json.load(file)

# # Créer une nouvelle ontologie
# onto = get_ontology("http://example.org/onto/pokemon")

# with onto:
#     class Pokemon(Thing):
#         pass

#     class Type(Thing):
#         pass
    
#     class Stat(Thing):
#         pass
    
#     class Sensi(Thing):
#         pass

#     class Attack(Thing):
#         pass

#     class hasType(ObjectProperty):
#         domain = [Pokemon]
#         range = [Type]
    
#     class hasStat(ObjectProperty):
#         domain = [Pokemon]
#         range = [Stat]
        
#     class hasSensi(ObjectProperty):
#         domain = [Pokemon]
#         range = [Sensi]

#     class hasAttack(ObjectProperty):
#         domain = [Pokemon]
#         range = [Attack]

#     class hasAttackType(ObjectProperty):
#         domain = [Attack]
#         range = [Type]

#     class nom(DataProperty, FunctionalProperty):
#         domain = [Pokemon, Type, Stat, Sensi, Attack]
#         range = [str]
    
#     class categorie(DataProperty, FunctionalProperty):
#         domain = [Attack]
#         range = [str]

#     class puissance(DataProperty, FunctionalProperty):
#         domain = [Attack]
#         range = [int]

#     class precision(DataProperty, FunctionalProperty):
#         domain = [Attack]
#         range = [int]

#     class pp(DataProperty, FunctionalProperty):
#         domain = [Attack]
#         range = [int]


#     class sensiType(ObjectProperty):
#         domain = [Sensi]
#         range = [Type]

#     class nom(DataProperty, FunctionalProperty):
#         domain = [Pokemon, Type, Stat, Sensi]
#         range = [str]
    
#     class valeur(DataProperty, FunctionalProperty): 
#         domain = [Stat]
#         range = [int]

#     class valeurfloat(DataProperty, FunctionalProperty):
#         domain = [Sensi]
#         range = [float]

#     types = {}
#     attacks = {}

#     for pokemon_data in pokemons_data:
#         # Types du Pokémon
#         types_instances = []
#         for type_name in pokemon_data['types']:
#             type_name = unidecode(type_name)
#             if type_name not in types:
#                 types[type_name] = Type(nom=type_name)
#             types_instances.append(types[type_name])
        
#         pokemon_nom = unidecode(pokemon_data['nom']) 
#         pokemon_instance = Pokemon(nom=pokemon_nom)
        
#         # Association des types au Pokémon
#         for type_instance in types_instances:
#             pokemon_instance.hasType.append(type_instance)
        
        # # Statistiques du Pokémon
        # for stat_name, stat_value in pokemon_data['stats'].items():
        #     stat_name = unidecode(stat_name)  
        #     stat_instance = Stat(nom=stat_name, valeur=stat_value)
        #     pokemon_instance.hasStat.append(stat_instance)
            
        # # Sensibilités du Pokémon
        # for sensibilite_name, sensibilite_value in pokemon_data['sensibilities'].items():
        #     type_instance = types.get(unidecode(sensibilite_name), Type(nom=unidecode(sensibilite_name)))
        #     sensibilite_instance = Sensi(nom=unidecode(sensibilite_name), valeurfloat=sensibilite_value)
        #     sensibilite_instance.sensiType.append(type_instance)
        #     pokemon_instance.hasSensi.append(sensibilite_instance)

        # # Attaques du Pokémon
        # for attack_data in pokemon_data['attaques']:
        #     attack_key = unidecode(attack_data['Nom'])
        #     if attack_key not in attacks:
        #         attack_type_name = attack_data['Type']
        #         if attack_type_name:
        #             attack_type_name = unidecode(attack_type_name)
        #             if attack_type_name not in types:
        #                 attack_type_instance = Type(nom=attack_type_name)
        #                 types[attack_type_name] = attack_type_instance
        #             else:
        #                 attack_type_instance = types[attack_type_name]
        #             attack_instance = Attack(
        #                 nom=attack_key,
        #                 categorie=unidecode(attack_data['Catégorie']) if attack_data['Catégorie'] else "Inconnu",
        #                 puissance=attack_data['Puissance'] if attack_data['Puissance'] is not None else 0,
        #                 precision=attack_data['Précision'] if attack_data['Précision'] is not None else 0,
        #                 pp=attack_data['PP'] if attack_data['PP'] is not None else 0
        #             )
        #             attack_instance.hasAttackType.append(attack_type_instance)
        #         else:
        #             # Handle attacks without a type
        #             attack_instance = Attack(
        #                 nom=attack_key,
        #                 categorie="Inconnu",
        #                 puissance=0,
        #                 precision=0,
        #                 pp=0
        #             )
        #         attacks[attack_key] = attack_instance
        #     pokemon_instance.hasAttack.append(attacks[attack_key])

# onto.save(file="pokemon_ontology.owl", format="rdfxml")

# print("Ontologie créée et sauvegardée avec les données JSON.")


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Ontologie créée et sauvegardée avec les données JSON.


In [1]:
# # Ontologie Finale 

# from owlready2 import get_ontology, Thing, DataProperty, ObjectProperty, FunctionalProperty
# from unidecode import unidecode
# import json

# # Charger les données JSON
# with open('./data/pokemons.json', 'r', encoding='utf-8') as file:
#     pokemons_data = json.load(file)

# # Créer une nouvelle ontologie
# onto = get_ontology("http://example.org/onto/pokemon")

# with onto:
#     class Pokemon(Thing):
#         pass

#     class Type(Thing):
#         pass
    
#     class Stat(Thing):
#         pass
    
#     class Sensi(Thing):
#         pass

#     class Attack(Thing):
#         pass

#     class hasType(ObjectProperty):
#         domain = [Pokemon]
#         range = [Type]
    
#     class hasStat(ObjectProperty):
#         domain = [Pokemon]
#         range = [Stat]
        
#     class hasSensi(ObjectProperty):
#         domain = [Pokemon]
#         range = [Sensi]

#     class hasAttack(ObjectProperty):
#         domain = [Pokemon]
#         range = [Attack]
        
#     class nom(DataProperty, FunctionalProperty):
#          domain = [Pokemon, Type, Stat, Sensi, Attack]
#          range = [str]

#     class valeur(DataProperty, FunctionalProperty): 
#         domain = [Stat, Sensi]
#         range = [float]
    
#     class hasSensiType(ObjectProperty, FunctionalProperty):
#         domain = [Sensi]
#         range = [Type]
        
#     class hasAttackType(ObjectProperty):
#         domain = [Attack]
#         range = [Type]
    
#     class categorie(DataProperty, FunctionalProperty):
#         domain = [Attack]
#         range = [str]

#     class puissance(DataProperty, FunctionalProperty):
#         domain = [Attack]
#         range = [int]

#     class precision(DataProperty, FunctionalProperty):
#         domain = [Attack]
#         range = [int]

#     class pp(DataProperty, FunctionalProperty):
#         domain = [Attack]
#         range = [int]

#     # Pré-remplir les types pour réutilisation
#     # Pré-remplir les types pour réutilisation, y compris un type "Inconnu"
#     types = {name: Type(nom=name) for name in ["Normal", "Plante", "Feu", "Eau", "Electrik", "Glace", "Combat", "Poison", "Sol", "Vol", "Psy", "Insecte", "Roche", "Spectre", "Dragon", "Tenebres", "Acier", "Fee"]}
#     types["Inconnu"] = Type(nom="Inconnu")  # Ajout du type "Inconnu" pour gérer les cas où le type est absent

#     stats_cache = {}
#     sensi_cache = {}
#     attacks = {}

#     # Gérer la création des Pokémons et leurs caractéristiques
#     for pokemon_data in pokemons_data:
#         pokemon_nom = unidecode(pokemon_data['nom']) 
#         pokemon_instance = Pokemon(nom=pokemon_nom)

#         # Attribuer les types aux Pokémon
#         for type_name in pokemon_data['types']:
#             pokemon_instance.hasType.append(types[unidecode(type_name)])

#         # Gérer les statistiques
#         for stat_name, stat_value in pokemon_data['stats'].items():
#             key = (unidecode(stat_name), float(stat_value))
#             if key not in stats_cache:
#                 stats_cache[key] = Stat(nom=key[0], valeur=key[1])
#             pokemon_instance.hasStat.append(stats_cache[key])
            
#         # Gérer les sensibilités
#         for sensi_name, sensi_value in pokemon_data['sensibilities'].items():
#             # Clé pour le cache basée uniquement sur le type (sans nom)
#             type_ref = types[unidecode(sensi_name)]
#             key = (type_ref, float(sensi_value))  
#             if key not in sensi_cache:
#                 sensi_instance = Sensi(valeur=key[1])
#                 sensi_instance.hasSensiType = type_ref  
#                 sensi_cache[key] = sensi_instance
#             pokemon_instance.hasSensi.append(sensi_cache[key])

#         # Attaques du Pokémon
#         for attack_data in pokemon_data['attaques']:
#             # Définir les valeurs par défaut pour les données manquantes
#             attack_type = unidecode(attack_data['Type']) if attack_data['Type'] else "Inconnu"
#             attack_categorie = unidecode(attack_data['Catégorie']) if attack_data['Catégorie'] else "Inconnu"
#             attack_puissance = attack_data['Puissance'] if attack_data['Puissance'] is not None else 0
#             attack_precision = attack_data['Précision'] if attack_data['Précision'] is not None else 0
#             attack_pp = attack_data['PP'] if attack_data['PP'] is not None else 0
            
#             # Clé du cache basée sur les attributs fonctionnels essentiels
#             attack_key = (attack_type, attack_categorie, attack_puissance, attack_precision, attack_pp)
            
#             if attack_key not in attacks:
#                 attack_type_instance = types[attack_type]
#                 attack_instance = Attack(
#                     nom=unidecode(attack_data['Nom']),
#                     categorie=attack_categorie,
#                     puissance=attack_puissance,
#                     precision=attack_precision,
#                     pp=attack_pp
#                 )
#                 attack_instance.hasAttackType.append(attack_type_instance)
#                 attacks[attack_key] = attack_instance
#             else:
#                 # Réutiliser l'instance existante pour éviter les redondances
#                 attack_instance = attacks[attack_key]
#                 # Si vous avez besoin de gérer différents noms pour la même configuration d'attaque,
#                 # vous pouvez envisager d'ajouter une propriété pour stocker les noms alternatifs
#                 # ou de modifier l'instance pour inclure le nouveau nom si c'est pertinent.

#             pokemon_instance.hasAttack.append(attack_instance)


            
#     onto.save(file="pokemon_ontology.owl", format="rdfxml")
#     print("Ontology created and saved.")


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Ontology created and saved.


In [1]:
# Ontologie Finale 

from owlready2 import get_ontology, Thing, DataProperty, ObjectProperty, FunctionalProperty
from unidecode import unidecode
import json

# Charger les données JSON
with open('./data/pokemons.json', 'r', encoding='utf-8') as file:
    pokemons_data = json.load(file)

# Créer une nouvelle ontologie
onto = get_ontology("http://example.org/onto/pokemon")

with onto:
    class Pokemon(Thing):
        pass

    class Type(Thing):
        pass
    
    class Stat(Thing):
        pass
    
    class Sensi(Thing):
        pass

    class Attack(Thing):
        pass

    class hasType(ObjectProperty):
        domain = [Pokemon]
        range = [Type]
    
    class hasStat(ObjectProperty):
        domain = [Pokemon]
        range = [Stat]
        
    class hasSensi(ObjectProperty):
        domain = [Pokemon]
        range = [Sensi]

    class hasAttack(ObjectProperty):
        domain = [Pokemon]
        range = [Attack]
        
    class nom(DataProperty, FunctionalProperty):
         domain = [Pokemon, Type, Stat, Sensi, Attack]
         range = [str]

    class valeur(DataProperty, FunctionalProperty): 
        domain = [Stat, Sensi]
        range = [float]
    
    class hasSensiType(ObjectProperty, FunctionalProperty):
        domain = [Sensi]
        range = [Type]
        
    class hasAttackType(ObjectProperty):
        domain = [Attack]
        range = [Type]
    
    class categorie(DataProperty, FunctionalProperty):
        domain = [Attack]
        range = [str]

    class puissance(DataProperty, FunctionalProperty):
        domain = [Attack]
        range = [int]

    class precision(DataProperty, FunctionalProperty):
        domain = [Attack]
        range = [int]

    class pp(DataProperty, FunctionalProperty):
        domain = [Attack]
        range = [int]

    # Pré-remplir les types pour réutilisation
    # Pré-remplir les types pour réutilisation, y compris un type "Inconnu"
    types = {name: Type(nom=name) for name in ["Normal", "Plante", "Feu", "Eau", "Electrik", "Glace", "Combat", "Poison", "Sol", "Vol", "Psy", "Insecte", "Roche", "Spectre", "Dragon", "Tenebres", "Acier", "Fee"]}
    types["Inconnu"] = Type(nom="Inconnu")  # Ajout du type "Inconnu" pour gérer les cas où le type est absent

    stats_cache = {}
    sensi_cache = {}
    attacks = {}

    # Gérer la création des Pokémons et leurs caractéristiques
    for pokemon_data in pokemons_data:
        pokemon_nom = unidecode(pokemon_data['nom']) 
        pokemon_instance = Pokemon(nom=pokemon_nom)

        # Attribuer les types aux Pokémon
        for type_name in pokemon_data['types']:
            pokemon_instance.hasType.append(types[unidecode(type_name)])

        # Gérer les statistiques
        for stat_name, stat_value in pokemon_data['stats'].items():
            key = (unidecode(stat_name), float(stat_value))
            if key not in stats_cache:
                stats_cache[key] = Stat(nom=key[0], valeur=key[1])
            pokemon_instance.hasStat.append(stats_cache[key])
            
        # Gérer les sensibilités
        for sensi_name, sensi_value in pokemon_data['sensibilities'].items():
            # Clé pour le cache basée uniquement sur le type (sans nom)
            type_ref = types[unidecode(sensi_name)]
            key = (type_ref, float(sensi_value))  
            if key not in sensi_cache:
                sensi_instance = Sensi(valeur=key[1])
                sensi_instance.hasSensiType = type_ref  
                sensi_cache[key] = sensi_instance
            pokemon_instance.hasSensi.append(sensi_cache[key])

        # Attaques du Pokémon
        for attack_data in pokemon_data['attaques']:
            attack_key = unidecode(attack_data['Nom'])
            if attack_key not in attacks:
                attack_type_name = attack_data['Type']
                if attack_type_name:
                    attack_type_name = unidecode(attack_type_name)
                    if attack_type_name not in types:
                        attack_type_instance = Type(nom=attack_type_name)
                        types[attack_type_name] = attack_type_instance
                    else:
                        attack_type_instance = types[attack_type_name]
                    attack_instance = Attack(
                        nom=attack_key,
                        categorie=unidecode(attack_data['Catégorie']) if attack_data['Catégorie'] else "Inconnu",
                        puissance=attack_data['Puissance'] if attack_data['Puissance'] is not None else 0,
                        precision=attack_data['Précision'] if attack_data['Précision'] is not None else 0,
                        pp=attack_data['PP'] if attack_data['PP'] is not None else 0
                    )
                    attack_instance.hasAttackType.append(attack_type_instance)
                else:
                    # Handle attacks without a type
                    attack_instance = Attack(
                        nom=attack_key,
                        categorie="Inconnu",
                        puissance=0,
                        precision=0,
                        pp=0
                    )
                attacks[attack_key] = attack_instance
            pokemon_instance.hasAttack.append(attacks[attack_key])



            
    onto.save(file="pokemon_ontology.owl", format="rdfxml")
    print("Ontology created and saved.")


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Ontology created and saved.


In [2]:
import json
from owlready2 import get_ontology, sync_reasoner, default_world

# Charger l'ontologie à partir du fichier sauvegardé
onto_path = "./pokemon_ontology.owl" 
onto = get_ontology(onto_path).load()

# Utiliser un moteur de raisonnement pour inférer de nouvelles connaissances
sync_reasoner()


* Owlready2 * Running HermiT...
    java -Xmx2000M -cp c:\Users\basti\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit;c:\Users\basti\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/basti/AppData/Local/Temp/tmptrynmfm2
* Owlready2 * HermiT took 3.572336435317993 seconds
* Owlready * Reparenting pokemon.attack395: {pokemon.Attack} => {pokemon.Pokemon, pokemon.Sensi, pokemon.Stat, pokemon.Attack, pokemon.Type}
* Owlready * Reparenting pokemon.stat311: {pokemon.Stat} => {pokemon.Pokemon, pokemon.Sensi, pokemon.Stat, pokemon.Attack, pokemon.Type}
* Owlready * Reparenting pokemon.pokemon314: {pokemon.Pokemon} => {pokemon.Pokemon, pokemon.Sensi, pokemon.Stat, pokemon.Attack, pokemon.Type}
* Owlready * Reparenting pokemon.attack396: {pokemon.Attack} => {pokemon.Pokemon, pokemon.Sensi, pokemon.Stat, pokemon.Attack, pokemon.Type}
* Owlready * Reparenting pokemon.st

In [6]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX onto: <http://example.org/onto/pokemon#>
SELECT DISTINCT ?pokemon WHERE {
    ?pokemon rdf:type onto:Pokemon .
    ?pokemon onto:hasType ?type .
    ?type onto:nom "Feu" .
}
"""

results = list(default_world.sparql(query))

for result in results:
    print(f"Pokémon de type Feu : {result[0].nom}")


Pokémon de type Feu : Salameche
Pokémon de type Feu : Reptincel
Pokémon de type Feu : Dracaufeu
Pokémon de type Feu : Goupix
Pokémon de type Feu : Feunard
Pokémon de type Feu : Caninos
Pokémon de type Feu : Arcanin
Pokémon de type Feu : Ponyta
Pokémon de type Feu : Galopa
Pokémon de type Feu : Magmar
Pokémon de type Feu : Pyroli
Pokémon de type Feu : Sulfura
Pokémon de type Feu : Ouisticram
Pokémon de type Feu : Chimpenfeu
Pokémon de type Feu : Simiabraz
Pokémon de type Feu : Maganon
Pokémon de type Feu : Heatran
Pokémon de type Feu : Hericendre
Pokémon de type Feu : Feurisson
Pokémon de type Feu : Typhlosion
Pokémon de type Feu : Limagma
Pokémon de type Feu : Volcaropod
Pokémon de type Feu : Malosse
Pokémon de type Feu : Demolosse
Pokémon de type Feu : Magby
Pokémon de type Feu : Entei
Pokémon de type Feu : Ho-Oh
Pokémon de type Feu : Poussifeu
Pokémon de type Feu : Galifeu
Pokémon de type Feu : Brasegali
Pokémon de type Feu : Chamallot
Pokémon de type Feu : Camerupt
Pokémon de type F

In [5]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX onto: <http://example.org/onto/pokemon#>
SELECT DISTINCT ?otherPokemon WHERE {
    ?reptincel onto:nom "Reptincel" .
    ?reptincel onto:hasType ?reptincelType .
    ?sensi onto:sensiType ?reptincelType .
    ?sensi onto:valeurfloat ?value .
    FILTER(?value >= 4) .
    ?otherPokemon onto:hasSensi ?sensi .
}
"""

# Exécution de la requête et récupération des résultats
results = list(default_world.sparql(query))

# Affichage des résultats
for result in results:
    pokemon = result[0]
    print(f"Pokémon sensible au type de Reptincel : {pokemon.nom}")


ValueError: No existing entity for IRI 'http://example.org/onto/pokemon#sensiType'! (use error_on_undefined_entities=False to accept unknown entities in SPARQL queries)

Test joueur vs joueur

https://www.pokepedia.fr/Olga_(Conseil_4)


In [18]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX onto: <http://example.org/onto/pokemon#>

SELECT DISTINCT ?pokemon ?typeNom WHERE {
  VALUES ?pokemonNom { "Lamantine" "Crustabri" "Cochignon" "Lokhlass" "Lippoutou" }
  ?pokemon onto:nom ?pokemonNom .
  ?pokemon onto:hasType ?type .
  ?type onto:nom ?typeNom .
}
"""

results = list(default_world.sparql(query))

for result in results:
    pokemon, typeNom = result
    print(f"{pokemon} est de type {typeNom}")


pokemon.pokemon328 est de type Glace
pokemon.pokemon328 est de type Sol
pokemon.pokemon91 est de type Eau
pokemon.pokemon91 est de type Glace
pokemon.pokemon87 est de type Eau
pokemon.pokemon87 est de type Glace
pokemon.pokemon124 est de type Glace
pokemon.pokemon124 est de type Psy
pokemon.pokemon131 est de type Eau
pokemon.pokemon131 est de type Glace


In [3]:
# query = """
# PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
# PREFIX onto: <http://example.org/onto/pokemon#>

# SELECT DISTINCT ?nomPokemonEquipe ?nomPokemonAdverse WHERE {
#   VALUES ?nomPokemonAdverse { "Lamantine" "Crustabri" "Cochignon" "Lokhlass" "Lippoutou" }
#   VALUES ?nomPokemonEquipe { "Dracaufeu" "Dardargnan" "Roucarnage" "Raichu" "Nosferalto" }
  
#   ?pokemonAdverse onto:nom ?nomPokemonAdverse .
#   ?pokemonAdverse onto:hasSensi ?sensi .
  
#   ?pokemonEquipe onto:nom ?nomPokemonEquipe .
#   ?pokemonEquipe onto:hasType ?typePokemonEquipe .
  
#   ?sensi onto:sensiType ?typePokemonEquipe .
#   ?sensi onto:valeurfloat ?value .
  
#   FILTER(?value >= 2)
# }
# """

# # SELECT DISTINCT ?nomPokemonEquipe ?nomPokemonAdverse WHERE


# # Exécution de la requête et récupération des résultats
# results = list(default_world.sparql(query))

# # Organiser les résultats pour chaque Pokémon de votre équipe
# sensibilites = {}
# for nomPokemonEquipe, nomPokemonAdverse in results:
#     if nomPokemonEquipe not in sensibilites:
#         sensibilites[nomPokemonEquipe] = []
#     sensibilites[nomPokemonEquipe].append(nomPokemonAdverse)

# # Afficher les Pokémon adverses sensibles pour chaque Pokémon de votre équipe
# for pokemonEquipe, pokemonsAdverses in sensibilites.items():
#     print(f"{pokemonEquipe} a un avantage contre : {', '.join(pokemonsAdverses)}")




In [4]:
# Liste des Pokémon du joueur
pokemons_joueur = ["Dracaufeu", "Dardargnan", "Dracolosse", "Raichu", "Nosferalto"]

def meilleur_pokemon_contre(adversaire_nom, pokemons_joueur):
    # Conversion de la liste des Pokémon du joueur en chaîne pour la requête SPARQL
    pokemons_joueur_filtre = ', '.join(f'"{p}"' for p in pokemons_joueur)
    
    query = f"""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX onto: <http://example.org/onto/pokemon#>
    SELECT DISTINCT ?pokemonJoueur ?typeJoueurNom ?valeurSensi
    WHERE {{
        ?pokemonJoueur onto:nom ?nomJoueur .
        FILTER (?nomJoueur IN ({pokemons_joueur_filtre})) .
        ?pokemonJoueur onto:hasType ?typeJoueur .
        ?typeJoueur onto:nom ?typeJoueurNom .
        ?adversaire onto:nom "{adversaire_nom}" .
        ?adversaire onto:hasSensi ?sensiAdversaire .
        ?sensiAdversaire onto:hasSensiType ?typeJoueur .
        ?sensiAdversaire onto:valeur ?valeurSensi .
    }} ORDER BY DESC(?valeurSensi)
    """
    
    # Exécuter la requête
    resultats = default_world.sparql(query)
    
    # Afficher le meilleur Pokémon
    for resultat in resultats:
        pokemon_nom = resultat[0].nom if resultat[0] else "Inconnu"
        type_nom = resultat[1] if resultat[1] else "Inconnu"
        efficacite = resultat[2] if resultat[2] else "Inconnue"
        print(f"Contre {adversaire_nom}, utilisez {pokemon_nom} avec une efficacité de {efficacite} grâce à son type {type_nom}.")
        break

# Exécuter la fonction pour chaque Pokémon adverse
for pokemon_adverse in ["Lamantine", "Crustabri", "Hyporoi", "Lokhlass", "Lippoutou"]:
    meilleur_pokemon_contre(pokemon_adverse, pokemons_joueur)


Contre Lamantine, utilisez Raichu avec une efficacité de 2.0 grâce à son type Electrik.
Contre Crustabri, utilisez Raichu avec une efficacité de 2.0 grâce à son type Electrik.
Contre Hyporoi, utilisez Dracolosse avec une efficacité de 2.0 grâce à son type Dragon.
Contre Lokhlass, utilisez Raichu avec une efficacité de 2.0 grâce à son type Electrik.
Contre Lippoutou, utilisez Dracaufeu avec une efficacité de 2.0 grâce à son type Feu.


In [5]:
from owlready2 import default_world

def meilleur_pokemon_contre(adversaire_nom, pokemons_joueur):
    pokemons_joueur_filtre = ', '.join([f'"{p}"' for p in pokemons_joueur])

    query = f"""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX onto: <http://example.org/onto/pokemon#>
    SELECT DISTINCT ?pokemonJoueur ?typeJoueur
    WHERE {{
        ?pokemonJoueur onto:nom ?nomJoueur .
        FILTER (?nomJoueur IN ({pokemons_joueur_filtre})) .
        ?pokemonJoueur onto:hasType ?typeJoueur .
        ?adversaire onto:nom "{adversaire_nom}" .
        ?adversaire onto:hasType ?advType .
        OPTIONAL {{
            ?pokemonJoueur onto:hasSensi ?sensiJoueur .
            ?sensiJoueur onto:sensiType ?advType .
            ?sensiJoueur onto:valeurfloat ?sensiVuln
        }}
        FILTER (COALESCE(?sensiVuln, 0) <= 1)  # Exclure les Pokémon vulnérables
    }}
    """

    results = default_world.sparql(query)
    print(f"Contre {adversaire_nom}, les Pokémon recommandés avec leurs types respectifs sont :")
    for result in results:
        pokemon = result[0].name if hasattr(result[0], 'name') else result[0]
        type_pokemon = result[1].name if hasattr(result[1], 'name') else result[1]
        print(f" - Pokémon: {pokemon} (Type: {type_pokemon})")

# Exécuter la fonction pour chaque Pokémon adverse
for pokemon_adverse in ["Lamantine", "Crustabri", "Hyporoi", "Lokhlass", "Lippoutou"]:
    meilleur_pokemon_contre(pokemon_adverse, pokemons_joueur)


Contre Lamantine, les Pokémon recommandés avec leurs types respectifs sont :
 - Pokémon: pokemon15 (Type: type179)
 - Pokémon: pokemon15 (Type: type2)
 - Pokémon: pokemon6 (Type: type61)
 - Pokémon: pokemon6 (Type: type85)
 - Pokémon: pokemon149 (Type: type80)
 - Pokémon: pokemon149 (Type: type85)
 - Pokémon: pokemon42 (Type: type2)
 - Pokémon: pokemon42 (Type: type85)
 - Pokémon: pokemon26 (Type: type450)
Contre Crustabri, les Pokémon recommandés avec leurs types respectifs sont :
 - Pokémon: pokemon15 (Type: type179)
 - Pokémon: pokemon15 (Type: type2)
 - Pokémon: pokemon6 (Type: type61)
 - Pokémon: pokemon6 (Type: type85)
 - Pokémon: pokemon149 (Type: type80)
 - Pokémon: pokemon149 (Type: type85)
 - Pokémon: pokemon42 (Type: type2)
 - Pokémon: pokemon42 (Type: type85)
 - Pokémon: pokemon26 (Type: type450)
Contre Hyporoi, les Pokémon recommandés avec leurs types respectifs sont :
 - Pokémon: pokemon15 (Type: type179)
 - Pokémon: pokemon15 (Type: type2)
 - Pokémon: pokemon6 (Type: typ

# Test avec les attaques 

In [11]:
def meilleur_pokemon_contre(adversaire_nom, pokemons_joueur):
    pokemons_joueur_filtre = ', '.join([f'"{p}"' for p in pokemons_joueur])

    query = f"""
    PREFIX onto: <http://example.org/onto/pokemon#>
    SELECT DISTINCT ?pokemonNom ?attackName ?attackType ?power ?sensiValue
    WHERE {{
        ?pokemon onto:nom ?pokemonNom .
        FILTER (?pokemonNom IN ({pokemons_joueur_filtre})) .
        ?pokemon onto:hasAttack ?attack .
        ?attack onto:nom ?attackName .
        ?attack onto:hasAttackType ?typeAttaque .
        ?typeAttaque onto:nom ?attackType .
        ?attack onto:puissance ?power .
        ?adversaire onto:nom "{adversaire_nom}" .
        ?adversaire onto:hasSensi ?sensiAdv .
        ?sensiAdv onto:hasSensiType ?typeAttaque .
        ?sensiAdv onto:valeur ?sensiValue .
        FILTER (?sensiValue >= 2) .
        
        # Vérifier que le Pokémon n'est pas vulnérable à aucun type de l'adversaire
        FILTER NOT EXISTS {{
            ?adversaire onto:hasType ?advType .
            ?pokemon onto:hasSensi ?pokemonSensi .
            ?pokemonSensi onto:hasSensiType ?advType .
            ?pokemonSensi onto:valeur ?sensiVuln .
            FILTER (?sensiVuln > 1) .
        }}
    }}
    ORDER BY DESC(?sensiValue) DESC(?power)
    LIMIT 3
    """

    results = default_world.sparql(query)
    print(f"Contre {adversaire_nom}, les meilleures attaques à utiliser sont :")
    for result in results:
        pokemon_nom = result[0] if result[0] else "Inconnu"
        attack_name = result[1] if result[1] else "Inconnue"
        attack_type = result[2] if result[2] else "Type Inconnu"
        power = result[3] if result[3] else "Puissance Inconnue"
        sensiValue = result[4] if result[4] else "Efficacité Inconnue"
        print(f" - {pokemon_nom} avec l'attaque '{attack_name}' (Type: {attack_type}) ayant une puissance de {power} et une efficacité de {sensiValue}.")

# Exécution de la fonction pour chaque adversaire
for adversaire in ["Lamantine", "Crustabri", "Hyporoi", "Lokhlass", "Lippoutou"]:
    meilleur_pokemon_contre(adversaire, pokemons_joueur)


Contre Lamantine, les meilleures attaques à utiliser sont :
 - Raichu avec l'attaque 'Electacle' (Type: Electrik) ayant une puissance de 120 et une efficacité de 2.0.
 - Raichu avec l'attaque 'Fatal-Foudre' (Type: Electrik) ayant une puissance de 110 et une efficacité de 2.0.
 - Raichu avec l'attaque 'Tonnerre' (Type: Electrik) ayant une puissance de 90 et une efficacité de 2.0.
Contre Crustabri, les meilleures attaques à utiliser sont :
 - Raichu avec l'attaque 'Electacle' (Type: Electrik) ayant une puissance de 120 et une efficacité de 2.0.
 - Raichu avec l'attaque 'Fatal-Foudre' (Type: Electrik) ayant une puissance de 110 et une efficacité de 2.0.
 - Raichu avec l'attaque 'Tonnerre' (Type: Electrik) ayant une puissance de 90 et une efficacité de 2.0.
Contre Hyporoi, les meilleures attaques à utiliser sont :
 - Raichu avec l'attaque 'Calinerie' (Type: Fee) ayant une puissance de 90 et une efficacité de 2.0.
 - Raichu avec l'attaque 'Voix Enjoleuse' (Type: Fee) ayant une puissance de 

from owlready2 import default_world

def meilleur_pokemon_contre(adversaire_nom, pokemons_joueur):
    pokemons_joueur_filtre = ', '.join([f'"{p}"' for p in pokemons_joueur])

    query = f"""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX onto: <http://example.org/onto/pokemon#>
    SELECT DISTINCT ?pokemonNom ?attackName ?attackType ?power ?sensiValue ?sensiVuln
    WHERE {{
        ?pokemon onto:nom ?pokemonNom .
        FILTER (?pokemonNom IN ({pokemons_joueur_filtre})) .
        ?pokemon onto:hasAttack ?attack .
        ?attack onto:nom ?attackName .
        ?attack onto:hasAttackType ?type .
        ?type onto:nom ?attackType .
        ?attack onto:puissance ?power .
        ?adversaire onto:nom "{adversaire_nom}" .
        ?adversaire onto:hasSensi ?sensiAdv .
        ?sensiAdv onto:sensiType ?type .
        ?sensiAdv onto:valeurfloat ?sensiValue .
        OPTIONAL {{
            ?pokemon onto:hasSensi ?sensiPokemon .
            ?adversaire onto:hasType ?advType .  # Fixing to check against all adversary types
            ?sensiPokemon onto:sensiType ?advType .  # Correct variable to compare with adversary types
            ?sensiPokemon onto:valeurfloat ?sensiVuln
        }}
        FILTER (COALESCE(?sensiVuln, 0) <= 1)  # Ensure Pokémon is not vulnerable
    }}
    ORDER BY DESC(?sensiValue) DESC(?power)
    LIMIT 3
    """

    results = default_world.sparql(query)
    print(f"Contre {adversaire_nom}, les meilleures attaques à utiliser sont :")
    for result in results:
        print(f" - {result[0]} avec l'attaque '{result[1]}' (Type: {result[2]}) ayant une puissance de {result[3]} et une efficacité de {result[4]}.")

adversaires = ["Lamantine", "Crustabri", "Hyporoi", "Lokhlass", "Lippoutou"]
for adversaire in adversaires:
    meilleur_pokemon_contre(adversaire, pokemons_joueur)

-------------------------------------------------
Contre Lamantine, les meilleures attaques à utiliser sont :
 - Raichu avec l'attaque 'Electacle' (Type: Electrik) ayant une puissance de 120 et une efficacité de 2.
 - Raichu avec l'attaque 'Fatal-Foudre' (Type: Electrik) ayant une puissance de 110 et une efficacité de 2.
 - Raichu avec l'attaque 'Tonnerre' (Type: Electrik) ayant une puissance de 90 et une efficacité de 2.
Contre Crustabri, les meilleures attaques à utiliser sont :
 - Raichu avec l'attaque 'Electacle' (Type: Electrik) ayant une puissance de 120 et une efficacité de 2.
 - Raichu avec l'attaque 'Fatal-Foudre' (Type: Electrik) ayant une puissance de 110 et une efficacité de 2.
 - Raichu avec l'attaque 'Tonnerre' (Type: Electrik) ayant une puissance de 90 et une efficacité de 2.
Contre Hyporoi, les meilleures attaques à utiliser sont :
 - Dracolosse avec l'attaque 'Draco-Meteore' (Type: Dragon) ayant une puissance de 130 et une efficacité de 2.
 - Dracaufeu avec l'attaque 'Colere' (Type: Dragon) ayant une puissance de 120 et une efficacité de 2.
 - Dracolosse avec l'attaque 'Colere' (Type: Dragon) ayant une puissance de 120 et une efficacité de 2.
Contre Lokhlass, les meilleures attaques à utiliser sont :
 - Raichu avec l'attaque 'Electacle' (Type: Electrik) ayant une puissance de 120 et une efficacité de 2.
 - Raichu avec l'attaque 'Fatal-Foudre' (Type: Electrik) ayant une puissance de 110 et une efficacité de 2.
 - Raichu avec l'attaque 'Tonnerre' (Type: Electrik) ayant une puissance de 90 et une efficacité de 2.
Contre Lippoutou, les meilleures attaques à utiliser sont :
 - Dracaufeu avec l'attaque 'Rafale Feu' (Type: Feu) ayant une puissance de 150 et une efficacité de 2.
 - Dracaufeu avec l'attaque 'Boutefeu' (Type: Feu) ayant une puissance de 120 et une efficacité de 2.
 - Dracaufeu avec l'attaque 'Feu d'Enfer' (Type: Feu) ayant une puissance de 100 et une efficacité de 2.

In [48]:
from owlready2 import default_world

def meilleur_pokemon_contre(adversaire_nom, pokemons_joueur):
    pokemons_joueur_filtre = ', '.join([f'"{p}"' for p in pokemons_joueur])

    query = f"""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX onto: <http://example.org/onto/pokemon#>
    SELECT DISTINCT ?pokemonNom ?attackName ?attackType ?power
    WHERE {{
        ?pokemon onto:nom ?pokemonNom .
        FILTER (?pokemonNom IN ({pokemons_joueur_filtre})) .
        ?pokemon onto:hasAttack ?attack .
        ?attack onto:nom ?attackName .
        ?attack onto:hasAttackType ?type .
        ?type onto:nom ?attackType .
        ?attack onto:puissance ?power .
        ?adversaire onto:nom "{adversaire_nom}" .
        ?adversaire onto:hasType ?advType .
        OPTIONAL {{
            ?pokemon onto:hasSensi ?sensiPokemon .
            ?sensiPokemon onto:sensiType ?advType .
            ?sensiPokemon onto:valeurfloat ?sensiVuln
        }}
        FILTER (COALESCE(?sensiVuln, 0) <= 1)  # Ensure Pokémon is not vulnerable to any of the adversary's types
    }}
    ORDER BY DESC(?power)  # Simplified to focus on power and fix issues with ?sensiValue
    LIMIT 3
    """

    results = default_world.sparql(query)
    print(f"Contre {adversaire_nom}, les meilleures attaques à utiliser sont :")
    for result in results:
        print(f" - {result[0]} avec l'attaque '{result[1]}' (Type: {result[2]}) ayant une puissance de {result[3]}.")

adversaires = ["Lamantine", "Crustabri", "Hyporoi", "Lokhlass", "Lippoutou"]
for adversaire in adversaires:
    meilleur_pokemon_contre(adversaire, pokemons_joueur)


Contre Lamantine, les meilleures attaques à utiliser sont :
 - Dracaufeu avec l'attaque 'Rafale Feu' (Type: Feu) ayant une puissance de 150.
 - Dracolosse avec l'attaque 'Ultralaser' (Type: Normal) ayant une puissance de 150.
 - Nosferalto avec l'attaque 'Giga Impact' (Type: Normal) ayant une puissance de 150.
Contre Crustabri, les meilleures attaques à utiliser sont :
 - Dracaufeu avec l'attaque 'Rafale Feu' (Type: Feu) ayant une puissance de 150.
 - Dracolosse avec l'attaque 'Ultralaser' (Type: Normal) ayant une puissance de 150.
 - Nosferalto avec l'attaque 'Giga Impact' (Type: Normal) ayant une puissance de 150.
Contre Hyporoi, les meilleures attaques à utiliser sont :
 - Dracaufeu avec l'attaque 'Rafale Feu' (Type: Feu) ayant une puissance de 150.
 - Dracolosse avec l'attaque 'Ultralaser' (Type: Normal) ayant une puissance de 150.
 - Nosferalto avec l'attaque 'Giga Impact' (Type: Normal) ayant une puissance de 150.
Contre Lokhlass, les meilleures attaques à utiliser sont :
 - Drac

In [24]:
from owlready2 import default_world

# Assurons que l'ontologie est correctement chargée dans `onto`

# Définition de la requête SPARQL
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX onto: <http://example.org/onto/pokemon#>

SELECT DISTINCT ?attackName ?attackType ?power
WHERE {
  ?pokemon onto:nom "Dracaufeu" .
  ?pokemon onto:hasAttack ?attack .
  ?attack onto:nom ?attackName .
  ?attack onto:puissance ?power .
  ?attack onto:hasAttackType ?type .
  ?type onto:nom ?attackType
}
"""

# Exécuter la requête SPARQL
results = default_world.sparql(query)

# Afficher les résultats
for result in results:
    print(f"Nom de l'attaque: {result[0]}, Type: {result[1]}, Puissance: {result[2]}")


Nom de l'attaque: Lame d'Air, Type: Vol, Puissance: 75
Nom de l'attaque: Canicule, Type: Feu, Puissance: 95
Nom de l'attaque: Draco-Griffe, Type: Dragon, Puissance: 80
Nom de l'attaque: Griffe, Type: Normal, Puissance: 40
Nom de l'attaque: Rugissement, Type: Normal, Puissance: 0
Nom de l'attaque: Flammeche, Type: Feu, Puissance: 40
Nom de l'attaque: Brouillard, Type: Normal, Puissance: 0
Nom de l'attaque: Draco-Souffle, Type: Dragon, Puissance: 60
Nom de l'attaque: Crocs Feu, Type: Feu, Puissance: 65
Nom de l'attaque: Tranche, Type: Normal, Puissance: 70
Nom de l'attaque: Lance-Flammes, Type: Feu, Puissance: 90
Nom de l'attaque: Grimace, Type: Normal, Puissance: 0
Nom de l'attaque: Danse Flammes, Type: Feu, Puissance: 35
Nom de l'attaque: Feu d'Enfer, Type: Feu, Puissance: 100
Nom de l'attaque: Boutefeu, Type: Feu, Puissance: 120
Nom de l'attaque: Pouvoir Antique, Type: Roche, Puissance: 60
Nom de l'attaque: Cognobidon, Type: Normal, Puissance: 0
Nom de l'attaque: Morsure, Type: Tenebr

In [5]:
from owlready2 import default_world

# Liste des Pokémon du joueur
pokemons_joueur = ["Dracaufeu", "Dardargnan", "Dracolosse", "Raichu", "Nosferalto"]

def meilleur_pokemon_contre(adversaire_nom, pokemons_joueur):
    pokemons_joueur_filtre = ', '.join([f'"{p}"' for p in pokemons_joueur])

    query = f"""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX onto: <http://example.org/onto/pokemon#>
    SELECT DISTINCT ?pokemonNom ?attackName ?attackType ?power (MAX(?sensiVuln) as ?maxSensi)
    WHERE {{
        ?pokemon onto:nom ?pokemonNom .
        FILTER (?pokemonNom IN ({pokemons_joueur_filtre})) .
        ?pokemon onto:hasAttack ?attack .
        ?attack onto:nom ?attackName .
        ?attack onto:hasAttackType ?type .
        ?type onto:nom ?attackType .
        ?attack onto:puissance ?power .
        ?adversaire onto:nom "{adversaire_nom}" .
        ?adversaire onto:hasType ?advType .
        OPTIONAL {{
            ?pokemon onto:hasSensi ?sensiPokemon .
            ?sensiPokemon onto:sensiType ?advType .
            ?sensiPokemon onto:valeurfloat ?sensiVuln .
        }}
    }}
    GROUP BY ?pokemonNom ?attackName ?attackType ?power
    HAVING (COALESCE(MAX(?sensiVuln), 1) <= 1)
    ORDER BY DESC(?power)
    LIMIT 3
    """

    results = default_world.sparql(query)
    print(f"Contre {adversaire_nom}, les meilleures attaques à utiliser sont :")
    for result in results:
        print(f" - {result[0]} avec l'attaque '{result[1]}' (Type: {result[2]}) ayant une puissance de {result[3]}.")

adversaires = ["Lamantine", "Crustabri", "Hyporoi", "Lokhlass", "Lippoutou"]
for adversaire in adversaires:
    meilleur_pokemon_contre(adversaire, pokemons_joueur)


Contre Lamantine, les meilleures attaques à utiliser sont :
 - Dracaufeu avec l'attaque 'Rafale Feu' (Type: Feu) ayant une puissance de 150.
 - Raichu avec l'attaque 'Giga Impact' (Type: Normal) ayant une puissance de 150.
 - Raichu avec l'attaque 'Ultralaser' (Type: Normal) ayant une puissance de 150.
Contre Crustabri, les meilleures attaques à utiliser sont :
 - Dracaufeu avec l'attaque 'Rafale Feu' (Type: Feu) ayant une puissance de 150.
 - Raichu avec l'attaque 'Giga Impact' (Type: Normal) ayant une puissance de 150.
 - Raichu avec l'attaque 'Ultralaser' (Type: Normal) ayant une puissance de 150.
Contre Hyporoi, les meilleures attaques à utiliser sont :
 - Dracaufeu avec l'attaque 'Rafale Feu' (Type: Feu) ayant une puissance de 150.
 - Nosferalto avec l'attaque 'Giga Impact' (Type: Normal) ayant une puissance de 150.
 - Nosferalto avec l'attaque 'Ultralaser' (Type: Normal) ayant une puissance de 150.
Contre Lokhlass, les meilleures attaques à utiliser sont :
 - Dracaufeu avec l'att

In [15]:
def meilleur_pokemon_contre(adversaire_nom, pokemons_joueur):
    pokemons_joueur_filtre = ', '.join([f'"{p}"' for p in pokemons_joueur])

    query = f"""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX onto: <http://example.org/onto/pokemon#>
    SELECT DISTINCT ?pokemonNom ?attackName ?attackType ?power ?sensiValue
    WHERE {{
        ?pokemon onto:nom ?pokemonNom .
        FILTER (?pokemonNom IN ({pokemons_joueur_filtre})) .
        ?pokemon onto:hasAttack ?attack .
        ?attack onto:nom ?attackName .
        ?attack onto:hasAttackType ?type .
        ?type onto:nom ?attackType .
        ?attack onto:puissance ?power .
        ?adversaire onto:nom "{adversaire_nom}" .
        ?adversaire onto:hasType ?advType .
        OPTIONAL {{
            ?adversaire onto:hasSensi ?sensiAdv .
            ?sensiAdv onto:sensiType ?advType .
            ?sensiAdv onto:valeurfloat ?sensiValue .
            FILTER (?sensiValue >= 2)  # Ensure attack is effective
        }}
        FILTER NOT EXISTS {{
            ?pokemon onto:hasSensi ?sensiPokemon .
            ?sensiPokemon onto:sensiType ?advType .
            ?sensiPokemon onto:valeurfloat ?sensiVuln .
            FILTER (?sensiVuln > 1)
        }}
    }}
    ORDER BY DESC(?power) DESC(?sensiValue)
    LIMIT 3
    """

    results = default_world.sparql(query)
    print(f"Contre {adversaire_nom}, les meilleures attaques à utiliser sont :")
    for result in results:
        print(f" - {result[0]} avec l'attaque '{result[1]}' (Type: {result[2]}) ayant une puissance de {result[3]} et une efficacité de {result[4]}.")

adversaires = ["Lamantine", "Crustabri", "Hyporoi", "Lokhlass", "Lippoutou"]
for adversaire in adversaires:
    meilleur_pokemon_contre(adversaire, pokemons_joueur)


Contre Lamantine, les meilleures attaques à utiliser sont :
 - Raichu avec l'attaque 'Ultralaser' (Type: Normal) ayant une puissance de 150 et une efficacité de None.
 - Nosferalto avec l'attaque 'Ultralaser' (Type: Normal) ayant une puissance de 150 et une efficacité de None.
 - Dracolosse avec l'attaque 'Ultralaser' (Type: Normal) ayant une puissance de 150 et une efficacité de None.
Contre Crustabri, les meilleures attaques à utiliser sont :
 - Raichu avec l'attaque 'Ultralaser' (Type: Normal) ayant une puissance de 150 et une efficacité de None.
 - Nosferalto avec l'attaque 'Ultralaser' (Type: Normal) ayant une puissance de 150 et une efficacité de None.
 - Dracolosse avec l'attaque 'Ultralaser' (Type: Normal) ayant une puissance de 150 et une efficacité de None.
Contre Hyporoi, les meilleures attaques à utiliser sont :
 - Raichu avec l'attaque 'Ultralaser' (Type: Normal) ayant une puissance de 150 et une efficacité de 2.
 - Nosferalto avec l'attaque 'Ultralaser' (Type: Normal) aya

In [25]:
from owlready2 import get_ontology, default_world

def selectionner_meilleurs_pokemons(adversaire_nom, pokemons_joueur):
    pokemons_joueur_filtre = ', '.join([f'"{p}"' for p in pokemons_joueur])

    # Modifier la requête pour inclure les types adverses et les sensibilités
    query = f"""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX onto: <http://example.org/onto/pokemon#>

    SELECT DISTINCT ?pokemonNom ?typeAdversaire ?sensiValeur
    WHERE {{
        ?pokemon onto:nom ?pokemonNom .
        FILTER (?pokemonNom IN ({pokemons_joueur_filtre})) .
        ?adversaire onto:nom "{adversaire_nom}" .
        ?adversaire onto:hasType ?typeAdv .
        ?typeAdv onto:nom ?typeAdversaire .
        OPTIONAL {{
            ?pokemon onto:hasSensi ?sensi .
            ?sensi onto:sensiType ?typeAdv .
            ?sensi onto:valeurfloat ?sensiValeur .
        }}
        FILTER (?sensiValeur <= 1)  # Assurer que le Pokémon n'est pas vulnérable
    }}
    ORDER BY ?pokemonNom
    """

    results = default_world.sparql(query)
    if results:
        print(f"Meilleurs Pokémon pour combattre {adversaire_nom} :")
        last_pokemon = ""
        for result in results:
            pokemon_nom, type_adversaire, sensi_valeur = result
            if pokemon_nom != last_pokemon:
                if last_pokemon:  # Print a newline between different Pokémon
                    print()
                print(f"{pokemon_nom} est efficace contre {adversaire_nom} avec les sensibilités suivantes :")
                last_pokemon = pokemon_nom
            print(f" - Sensibilité à {type_adversaire}: {sensi_valeur if sensi_valeur else 'Aucune donnée'}")
        print()
    else:
        print(f"Aucun Pokémon de votre équipe n'est adapté pour combattre {adversaire_nom} sans risque de sensibilité élevée.")

# Liste de vos Pokémon
pokemons_joueur = ["Dracaufeu", "Dardargnan", "Dracolosse", "Raichu", "Nosferalto"]
# Nom de l'adversaire
adversaire_nom = "Hyporoi"

# Appel de la fonction
selectionner_meilleurs_pokemons(adversaire_nom, pokemons_joueur)


Meilleurs Pokémon pour combattre Hyporoi :
Dardargnan est efficace contre Hyporoi avec les sensibilités suivantes :
 - Sensibilité à Eau: 1
 - Sensibilité à Dragon: 1

Dracaufeu est efficace contre Hyporoi avec les sensibilités suivantes :
 - Sensibilité à Dragon: 1

Dracolosse est efficace contre Hyporoi avec les sensibilités suivantes :
 - Sensibilité à Eau: 0.5

Nosferalto est efficace contre Hyporoi avec les sensibilités suivantes :
 - Sensibilité à Eau: 1
 - Sensibilité à Dragon: 1

Raichu est efficace contre Hyporoi avec les sensibilités suivantes :
 - Sensibilité à Eau: 1
 - Sensibilité à Dragon: 1



In [14]:
from owlready2 import get_ontology, default_world

def afficher_sensibilites_equipe(pokemons_joueur):

    pokemons_joueur_filtre = ', '.join([f'"{p}"' for p in pokemons_joueur])
    
    query = f"""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX onto: <http://example.org/onto/pokemon#>

    SELECT DISTINCT ?pokemonNom ?typeNom ?sensiValeur
    WHERE {{
        ?pokemon onto:nom ?pokemonNom .
        FILTER (?pokemonNom IN ({pokemons_joueur_filtre})) .
        ?type a onto:Type .
        ?type onto:nom ?typeNom .
        OPTIONAL {{
            ?pokemon onto:hasSensi ?sensi .
            ?sensi onto:sensiType ?type .
            ?sensi onto:valeurfloat ?sensiValeur .
        }}
    }}
    """

    results = default_world.sparql(query)
    if results:
        print("Sensibilités de l'équipe à tous les types :")
        last_pokemon = ""
        for result in results:
            pokemon_nom, type_nom, sensi_valeur = result
            if pokemon_nom != last_pokemon:
                if last_pokemon:  # Print a newline between different Pokémon
                    print()
                print(f"{pokemon_nom} a les sensibilités suivantes :")
                last_pokemon = pokemon_nom
            print(f" - Sensibilité à {type_nom}: {sensi_valeur if sensi_valeur else 'Aucune donnée'}")
        print()
    else:
        print("Aucune donnée de sensibilité disponible pour l'équipe.")

# Liste de vos Pokémon
pokemons_joueur = ["Dracaufeu"]

# Appel de la fonction
afficher_sensibilites_equipe(pokemons_joueur)


Sensibilités de l'équipe à tous les types :
Dracaufeu a les sensibilités suivantes :
 - Sensibilité à Plante: 0.25
 - Sensibilité à Poison: 1
 - Sensibilité à Normal: Aucune donnée
 - Sensibilité à Plante: Aucune donnée
 - Sensibilité à Feu: Aucune donnée
 - Sensibilité à Eau: Aucune donnée
 - Sensibilité à Electrik: Aucune donnée
 - Sensibilité à Glace: Aucune donnée
 - Sensibilité à Combat: Aucune donnée
 - Sensibilité à Poison: Aucune donnée
 - Sensibilité à Sol: Aucune donnée
 - Sensibilité à Vol: Aucune donnée
 - Sensibilité à Psy: Aucune donnée
 - Sensibilité à Insecte: Aucune donnée
 - Sensibilité à Roche: Aucune donnée
 - Sensibilité à Spectre: Aucune donnée
 - Sensibilité à Dragon: Aucune donnée
 - Sensibilité à Tenebres: Aucune donnée
 - Sensibilité à Acier: Aucune donnée
 - Sensibilité à Fee: Aucune donnée
 - Sensibilité à Normal: 1
 - Sensibilité à Spectre: 1
 - Sensibilité à Psy: 1
 - Sensibilité à Fee: 0.5
 - Sensibilité à Feu: 0.5
 - Sensibilité à Dragon: 1
 - Sensibilit

In [6]:
from owlready2 import default_world

# Assurez-vous que votre ontologie est chargée
onto = get_ontology("http://example.org/onto/pokemon").load()

# La requête SPARQL
query = """
PREFIX : <http://example.org/onto/pokemon#>
SELECT DISTINCT ?typeName ?sensiValue
WHERE {
  ?pokemon :nom "Dracaufeu" .
  ?pokemon :hasSensi ?sensi .
  ?sensi :hasSensiType ?type .
  ?type :nom ?typeName .
  ?sensi :valeur ?sensiValue .
}
"""

# Exécuter la requête SPARQL
results = default_world.sparql(query)

# Afficher les résultats
for result in results:
    print(f"Type: {result[0]}, Sensibilité (Valeur): {result[1]}")


Type: Normal, Sensibilité (Valeur): 1.0
Type: Plante, Sensibilité (Valeur): 0.25
Type: Feu, Sensibilité (Valeur): 0.5
Type: Eau, Sensibilité (Valeur): 2.0
Type: Electrik, Sensibilité (Valeur): 2.0
Type: Glace, Sensibilité (Valeur): 1.0
Type: Combat, Sensibilité (Valeur): 0.5
Type: Poison, Sensibilité (Valeur): 1.0
Type: Sol, Sensibilité (Valeur): 0.0
Type: Vol, Sensibilité (Valeur): 1.0
Type: Psy, Sensibilité (Valeur): 1.0
Type: Insecte, Sensibilité (Valeur): 0.25
Type: Roche, Sensibilité (Valeur): 4.0
Type: Spectre, Sensibilité (Valeur): 1.0
Type: Dragon, Sensibilité (Valeur): 1.0
Type: Tenebres, Sensibilité (Valeur): 1.0
Type: Acier, Sensibilité (Valeur): 0.5
Type: Fee, Sensibilité (Valeur): 0.5


In [3]:
from owlready2 import default_world

# Assurez-vous que votre ontologie est chargée
onto = get_ontology("http://example.org/onto/pokemon").load()

# La requête SPARQL
query = """
PREFIX : <http://example.org/onto/pokemon#>
SELECT DISTINCT ?statName ?value
WHERE {
  ?pokemon :nom "Dracaufeu" .
  ?pokemon :hasStat ?stat .
  ?stat :nom ?statName .
  ?stat :valeur ?value .
}
"""

# Exécuter la requête SPARQL
results = default_world.sparql(query)

# Afficher les résultats
for result in results:
    print(f"Statistique: {result[0]}, Valeur: {result[1]}")


Statistique: PV, Valeur: 78.0
Statistique: Attaque, Valeur: 84.0
Statistique: Defense, Valeur: 78.0
Statistique: Attaque Speciale, Valeur: 109.0
Statistique: Defense Speciale, Valeur: 85.0
Statistique: Vitesse, Valeur: 100.0
Statistique: Special, Valeur: 85.0
